### Pipeline

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from workalendar.europe import Netherlands
import gc
import random

In [2]:
def process_data(file_path):
    # Load data
    df = pd.read_csv(file_path)

    # Filter for a smaller df and drop unnecessary columns
    df = df.head(100)
    df = df.drop('Unnamed: 0', axis=1)

    # Define helper functions
    def is_workday(date):
        return date.weekday() < 5

    def is_working_hours(time):
        return 9 <= time.hour < 17

    def is_not_holiday(date, holidays):
        return date not in holidays

    def get_dutch_holidays(year):
        cal = Netherlands()
        return {date: name for date, name in cal.holidays(year)}

    def generate_time_slots(start_date, end_date, interval='15min'):
        return pd.date_range(start=start_date, end=end_date, freq=interval)

    # Initialize Dutch calendar and get holidays
    def get_low_high_tariff():
        cal = Netherlands()
        year = 2023
        nl_holidays_2023 = cal.holidays(year)

        start_date = datetime(2023, 1, 1, 0, 0)
        end_date = datetime(2023, 12, 31, 23, 45)

        timestamps = []
        current_date = start_date
        while current_date <= end_date:
            timestamps.append(current_date)
            current_date += timedelta(minutes=15)

        high_tariff = []
        for timestamp in timestamps:
            if is_workday(timestamp) and is_working_hours(timestamp.time()) and is_not_holiday(timestamp, nl_holidays_2023):
                high_tariff.append(1)
            else:
                high_tariff.append(0)

        return pd.DataFrame({'Timestamp': timestamps, 'High_Tariff': high_tariff})

    high_low_tariff = get_low_high_tariff()

    high_low_tariff['Timestamp'] = pd.to_datetime(high_low_tariff['Timestamp'])

    # Extract date-time columns and RND_ID, Baseload_profile, Connection category columns
    date_time_cols = [col for col in df.columns if '2023' in col]
    id_cols = [col for col in df.columns if col not in date_time_cols]

    # Convert date-time columns to long format
    df = df.melt(id_vars=id_cols, value_vars=date_time_cols, var_name='Timestamp', value_name='Consumption')

    # Convert 'Timestamp' to string to ensure format consistency
    df['Timestamp'] = df['Timestamp'].astype(str)

    # Merge high-low tariff information with the main data
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    df = pd.merge(df, high_low_tariff, on='Timestamp', how='left')

    # Add time-based features
    df['hour'] = df['Timestamp'].dt.hour
    df['day_of_week'] = df['Timestamp'].dt.dayofweek
    df['month'] = df['Timestamp'].dt.month
    df['is_weekend'] = df['day_of_week'] >= 5

    # Compute monthly total, average, and maximum consumption
    monthly_total = df.groupby(['RND_ID', 'month'])['Consumption'].sum().reset_index()
    monthly_avg = df.groupby(['RND_ID', 'month'])['Consumption'].mean().reset_index()
    monthly_max = df.groupby(['RND_ID', 'month'])['Consumption'].max().reset_index()

    # Merge monthly total and average back to the main data
    df = pd.merge(df, monthly_total, on=['RND_ID', 'month'], suffixes=('', '_monthly_total'))
    df = pd.merge(df, monthly_avg, on=['RND_ID', 'month'], suffixes=('', '_monthly_avg'))
    df = pd.merge(df, monthly_max, on=['RND_ID', 'month'], suffixes=('', '_monthly_max'))

    # Compute annual maximum consumption
    annual_max = df.groupby(['BASELOAD_PROFILE', 'RND_ID'])['Consumption'].max().reset_index()
    df = pd.merge(df, annual_max, on=['BASELOAD_PROFILE', 'RND_ID'], suffixes=('', '_annual_max'))

    # Bin customers based on their maximum power consumption
    baseload_profiles = df['BASELOAD_PROFILE'].unique()
    num_bins = 5
    bin_labels, bin_edges = pd.qcut(df['Consumption_annual_max'], num_bins, labels=False, retbins=True)
    df['bin'] = bin_labels

    # Sort the resulting DataFrame
    df = df.sort_values(by='bin')

    # Define bin ranges and merge with the main data
    bin_ranges = pd.DataFrame({
        'bin': range(num_bins),
        'range_start': bin_edges[:-1],
        'range_end': bin_edges[1:]
    })
    df = df.merge(bin_ranges, on='bin', how='left')

    return df

In [3]:
# Example usage
file_path = "merged_dfs.csv"  # replace with your file path
df_processed = process_data(file_path)
df_processed.head()

,RND_ID,BASELOAD_PROFILE,CONNECTION_CATEGORY,Timestamp,Consumption,High_Tariff,hour,day_of_week,month,is_weekend,Consumption_monthly_total,Consumption_monthly_avg,Consumption_monthly_max,Consumption_annual_max,bin,range_start,range_end
0,4109,E3A,AC4B,2023-12-31 23:45:00,7.71,0,23,6,12,True,21328.84,7.166949,9.5,12.0,0,12.0,26.152
1,5864,PV,AC4B,2023-11-01 03:30:00,0.00,0,3,2,11,False,2528.00,0.219444,8.0,16.0,0,12.0,26.152
2,5864,PV,AC4B,2023-11-01 03:30:00,0.00,0,3,2,11,False,2528.00,0.219444,8.0,16.0,0,12.0,26.152
3,5864,PV,AC4B,2023-11-01 03:45:00,0.00,0,3,2,11,False,2528.00,0.219444,8.0,16.0,0,12.0,26.152
4,5864,PV,AC4B,2023-11-01 03:45:00,0.00,0,3,2,11,False,2528.00,0.219444,8.0,16.0,0,12.0,26.152


In [5]:
a = pd.read_csv('100_conditioned.csv', low_memory=False)
a.head()

,Unnamed: 0,RND_ID,BASELOAD_PROFILE,CONNECTION_CATEGORY,Timestamp,Consumption,High_Tariff,hour,day_of_week,month,is_weekend,Consumption_monthly_total,Consumption_monthly_avg,Consumption_monthly_max,Consumption_anual_max,bin,range_start,range_end
0,0,4109,E3A,AC4B,2023-12-31 23:45:00,7.71,0,23,6,12,True,21328.84,7.166949,9.5,12.0,0,12.0,26.152
1,1,5864,PV,AC4B,2023-11-01 03:30:00,0.00,0,3,2,11,False,2528.00,0.219444,8.0,16.0,0,12.0,26.152
2,2,5864,PV,AC4B,2023-11-01 03:30:00,0.00,0,3,2,11,False,2528.00,0.219444,8.0,16.0,0,12.0,26.152
3,3,5864,PV,AC4B,2023-11-01 03:45:00,0.00,0,3,2,11,False,2528.00,0.219444,8.0,16.0,0,12.0,26.152
4,4,5864,PV,AC4B,2023-11-01 03:45:00,0.00,0,3,2,11,False,2528.00,0.219444,8.0,16.0,0,12.0,26.152


#### Index Pipeline

In [6]:
def process_data_index(file_path, start_index=0, batch_size=100):
    # Load data
    df = pd.read_csv(file_path)

    # Filter for a smaller df starting from start_index
    df = df.iloc[start_index:start_index + batch_size]
    df = df.drop('Unnamed: 0', axis=1)

    # Filter for a smaller df and drop unnecessary columns
    #df = df.head(100)
    #df = df.drop('Unnamed: 0', axis=1)

    # Define helper functions
    def is_workday(date):
        return date.weekday() < 5

    def is_working_hours(time):
        return 9 <= time.hour < 17

    def is_not_holiday(date, holidays):
        return date not in holidays

    def get_dutch_holidays(year):
        cal = Netherlands()
        return {date: name for date, name in cal.holidays(year)}

    def generate_time_slots(start_date, end_date, interval='15min'):
        return pd.date_range(start=start_date, end=end_date, freq=interval)

    # Initialize Dutch calendar and get holidays
    def get_low_high_tariff():
        cal = Netherlands()
        year = 2023
        nl_holidays_2023 = cal.holidays(year)

        start_date = datetime(2023, 1, 1, 0, 0)
        end_date = datetime(2023, 12, 31, 23, 45)

        timestamps = []
        current_date = start_date
        while current_date <= end_date:
            timestamps.append(current_date)
            current_date += timedelta(minutes=15)

        high_tariff = []
        for timestamp in timestamps:
            if is_workday(timestamp) and is_working_hours(timestamp.time()) and is_not_holiday(timestamp, nl_holidays_2023):
                high_tariff.append(1)
            else:
                high_tariff.append(0)

        return pd.DataFrame({'Timestamp': timestamps, 'High_Tariff': high_tariff})

    high_low_tariff = get_low_high_tariff()

    high_low_tariff['Timestamp'] = pd.to_datetime(high_low_tariff['Timestamp'])

    # Extract date-time columns and RND_ID, Baseload_profile, Connection category columns
    date_time_cols = [col for col in df.columns if '2023' in col]
    id_cols = [col for col in df.columns if col not in date_time_cols]

    # Convert date-time columns to long format
    df = df.melt(id_vars=id_cols, value_vars=date_time_cols, var_name='Timestamp', value_name='Consumption')

    # Convert 'Timestamp' to string to ensure format consistency
    df['Timestamp'] = df['Timestamp'].astype(str)

    # Merge high-low tariff information with the main data
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    df = pd.merge(df, high_low_tariff, on='Timestamp', how='left')

    # Add time-based features
    df['hour'] = df['Timestamp'].dt.hour
    df['day_of_week'] = df['Timestamp'].dt.dayofweek
    df['month'] = df['Timestamp'].dt.month
    df['is_weekend'] = df['day_of_week'] >= 5

    # Compute monthly total, average, and maximum consumption
    monthly_total = df.groupby(['RND_ID', 'month'])['Consumption'].sum().reset_index()
    monthly_avg = df.groupby(['RND_ID', 'month'])['Consumption'].mean().reset_index()
    monthly_max = df.groupby(['RND_ID', 'month'])['Consumption'].max().reset_index()

    # Merge monthly total and average back to the main data
    df = pd.merge(df, monthly_total, on=['RND_ID', 'month'], suffixes=('', '_monthly_total'))
    df = pd.merge(df, monthly_avg, on=['RND_ID', 'month'], suffixes=('', '_monthly_avg'))
    df = pd.merge(df, monthly_max, on=['RND_ID', 'month'], suffixes=('', '_monthly_max'))

    # Compute annual maximum consumption
    annual_max = df.groupby(['BASELOAD_PROFILE', 'RND_ID'])['Consumption'].max().reset_index()
    df = pd.merge(df, annual_max, on=['BASELOAD_PROFILE', 'RND_ID'], suffixes=('', '_annual_max'))

    # Bin customers based on their maximum power consumption
    baseload_profiles = df['BASELOAD_PROFILE'].unique()
    num_bins = 5
    bin_labels, bin_edges = pd.qcut(df['Consumption_annual_max'], num_bins, labels=False, retbins=True)
    df['bin'] = bin_labels

    # Sort the resulting DataFrame
    df = df.sort_values(by='bin')

    # Define bin ranges and merge with the main data
    bin_ranges = pd.DataFrame({
        'bin': range(num_bins),
        'range_start': bin_edges[:-1],
        'range_end': bin_edges[1:]
    })
    df = df.merge(bin_ranges, on='bin', how='left')

    return df, start_index + batch_size

In [7]:
# Use the function to process the first batch of data
start_index = 0
file_path = "merged_dfs.csv"
df_processed_ind, next_start_index = process_data_index(file_path, start_index)

In [8]:
df_processed_ind.head()

,RND_ID,BASELOAD_PROFILE,CONNECTION_CATEGORY,Timestamp,Consumption,High_Tariff,hour,day_of_week,month,is_weekend,Consumption_monthly_total,Consumption_monthly_avg,Consumption_monthly_max,Consumption_annual_max,bin,range_start,range_end
0,4109,E3A,AC4B,2023-12-31 23:45:00,7.71,0,23,6,12,True,21328.84,7.166949,9.5,12.0,0,12.0,26.152
1,5864,PV,AC4B,2023-11-01 03:30:00,0.00,0,3,2,11,False,2528.00,0.219444,8.0,16.0,0,12.0,26.152
2,5864,PV,AC4B,2023-11-01 03:30:00,0.00,0,3,2,11,False,2528.00,0.219444,8.0,16.0,0,12.0,26.152
3,5864,PV,AC4B,2023-11-01 03:45:00,0.00,0,3,2,11,False,2528.00,0.219444,8.0,16.0,0,12.0,26.152
4,5864,PV,AC4B,2023-11-01 03:45:00,0.00,0,3,2,11,False,2528.00,0.219444,8.0,16.0,0,12.0,26.152


In [9]:
next_start_index

100

In [10]:
# Use next_start_index to process the next batch
df_processed_next, next_start_index_ = process_data_index(file_path, next_start_index)

In [11]:
df_processed_next.head()

,RND_ID,BASELOAD_PROFILE,CONNECTION_CATEGORY,Timestamp,Consumption,High_Tariff,hour,day_of_week,month,is_weekend,Consumption_monthly_total,Consumption_monthly_avg,Consumption_monthly_max,Consumption_annual_max,bin,range_start,range_end
0,30,E3A,AC4A,2023-10-01 18:15:00,0.09,0,18,6,10,True,-897.13,-0.301455,8.11,12.0,0,0.17,23.678
1,6741,E3D,AC4B,2023-09-01 07:00:00,1.04,0,7,4,9,False,13868.32,1.203847,5.59,8.0,0,0.17,23.678
2,6741,E3D,AC4B,2023-09-01 07:00:00,1.04,0,7,4,9,False,13868.32,1.203847,5.59,8.0,0,0.17,23.678
3,6741,E3D,AC4B,2023-09-01 07:15:00,2.04,0,7,4,9,False,13868.32,1.203847,5.59,8.0,0,0.17,23.678
4,6741,E3D,AC4B,2023-09-01 07:15:00,2.04,0,7,4,9,False,13868.32,1.203847,5.59,8.0,0,0.17,23.678


In [12]:
next_start_index_

200